In [0]:
dbutils.fs.help()

dbutils.fs provides utilities for working with FileSystems. Most methods in
this package can take either a DBFS path (e.g., "/foo" or "dbfs:/foo"), or
another FileSystem URI.

For more info about a method, use dbutils.fs.help("methodName") .

In notebooks, you can also use the %fs shorthand to access DBFS. The %fs shorthand maps
straightforwardly onto dbutils calls. For example, "%fs head --maxBytes=10000 /file/path"
translates into "dbutils.fs.head("/file/path", maxBytes = 10000)".
 mount mount(source: String, mountPoint: String, encryptionType: String = "", owner: String = null, extraConfigs: Map = Map.empty[String, String]): boolean -> Mounts the given source directory into DBFS at the given mount point mounts: Seq -> Displays information about what is mounted within DBFS refreshMounts: boolean -> Forces all machines in this cluster to refresh their mount cache, ensuring they receive the most recent information unmount(mountPoint: String): boolean -> Deletes a DBFS mount point updateMount(source: String, mountPoint: String, encryptionType: String = "", owner: String = null, extraConfigs: Map = Map.empty[String, String]): boolean -> Similar to mount(), but updates an existing mount point (if present) instead of creating a new one fsutils cp(from: String, to: String, recurse: boolean = false): boolean -> Copies a file or directory, possibly across FileSystems head(file: String, maxBytes: int = 65536): String -> Returns up to the first 'maxBytes' bytes of the given file as a String encoded in UTF-8 ls(dir: String): Seq -> Lists the contents of a directory mkdirs(dir: String): boolean -> Creates the given directory if it does not exist, also creating any necessary parent directories mv(from: String, to: String, recurse: boolean = false): boolean -> Moves a file or directory, possibly across FileSystems put(file: String, contents: String, overwrite: boolean = false): boolean -> Writes the given String out to a file, encoded in UTF-8 rm(dir: String, recurse: boolean = false): boolean -> Removes a file or directory

## Mount Data Lake Gen2

In [0]:
# Mount DataLake Gen 2 to Databricks

# gather relevant Service Principal keys
SPApplicationID = dbutils.secrets.get(scope='TestScope', key='SPAppID')
ServicePrincipalKey = dbutils.secrets.get(scope='TestScope', key='SPsecretkey')
DirectoryID = dbutils.secrets.get(scope='TestScope', key='SPDirID')


# Create configurations for connection
configs = {"fs.azure.account.auth.type": "OAuth",
           "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
           "fs.azure.account.oauth2.client.id": SPApplicationID,
           "fs.azure.account.oauth2.client.secret": ServicePrincipalKey,
           "fs.azure.account.oauth2.client.endpoint": DirectoryID}

# Mount datalakegen2 onto databricks dbfs
dbutils.fs.mount(
  source = "abfss://test@storageaccountdl1738.dfs.core.windows.net/",
  mount_point = "/mnt/Testadl",
  extra_configs = configs)


Out[17]: True

In [0]:
#check files in mount point

dbutils.fs.ls("/mnt/Testadl")

Out[18]: [FileInfo(path='dbfs:/mnt/Testadl/pokemon.csv', name='pokemon.csv', size=45254, modificationTime=1672997333000)]

In [0]:
# unmount storage system

dbutils.fs.unmount('/mnt//')

/mnt/TestMountBlobPython has been unmounted.
Out[16]: True

In [0]:
# read data from mounted adlgen2 storage

df = spark.read.format("csv")\
    .option("inferSchema", True) \
    .option("header", True) \
    .option("sep", ",") \
    .load("/mnt/Testadl/pokemon.csv")

df.show(10)

+---+----------+-------------+------+------+---------------+------+------+---+------+-------+-----+--------------+---------------+
| id|   species|generation_id|height|weight|base_experience|type_1|type_2| hp|attack|defense|speed|special-attack|special-defense|
+---+----------+-------------+------+------+---------------+------+------+---+------+-------+-----+--------------+---------------+
|  1| bulbasaur|            1|   0.7|   6.9|             64| grass|poison| 45|    49|     49|   45|            65|             65|
|  2|   ivysaur|            1|   1.0|  13.0|            142| grass|poison| 60|    62|     63|   60|            80|             80|
|  3|  venusaur|            1|   2.0| 100.0|            236| grass|poison| 80|    82|     83|   80|           100|            100|
|  4|charmander|            1|   0.6|   8.5|             62|  fire|  null| 39|    52|     43|   65|            60|             50|
|  5|charmeleon|            1|   1.1|  19.0|            142|  fire|  null| 58|    6

## Mounting Blob Storage

In [0]:
# Mount Blob Storage to Databricks DBFS

dbutils.fs.mount(
  source = "wasbs://test-blob@storageaccountblob.blob.core.windows.net",
  mount_point = "/mnt/TestMountBlobPython",
  extra_configs = {"fs.azure.account.key.storageaccountblob.blob.core.windows.net": dbutils.secrets.get(scope='testScope', key='blobAccountKey')}
)

Out[6]: True

In [0]:
#check files in mount point

dbutils.fs.ls("/mnt/TestMountBlobPython")

Out[21]: [FileInfo(path='dbfs:/mnt/TestMountBlobPython/201902-fordgobike-tripdata.csv', name='201902-fordgobike-tripdata.csv', size=39422395, modificationTime=1672766078000),
 FileInfo(path='dbfs:/mnt/TestMountBlobPython/sales_data.csv', name='sales_data.csv', size=22441174, modificationTime=1672766132000)]

In [0]:
# unmount storage system

dbutils.fs.unmount('/mnt/TestMountBlobPython')

/mnt/TestMountBlobPython has been unmounted.
Out[5]: True

In [0]:
# check to see if Account Keys from Azure Secret Vault is redacted

abd = dbutils.secrets.get(scope='testScope', key='blobAccountKey')

print(abd)

In [0]:
# Reading Data from the Mounted Blob Storage

df = spark.read.format("csv")\
    .option("inferSchema", True) \
    .option("header", True) \
    .option("sep", ",") \
    .load("/mnt/TestMountBlobPython/sales_data.csv")

df.show(10)

+---+--------+--------------------+----------------+----------+-------------------+--------------------+-----+------+--------------+----+
|_c0|Order ID|             Product|Quantity Ordered|Price Each|         Order Date|    Purchase Address|Month| Sales|          City|Hour|
+---+--------+--------------------+----------------+----------+-------------------+--------------------+-----+------+--------------+----+
|  0|  295665|  Macbook Pro Laptop|               1|    1700.0|2019-12-30 00:01:00|136 Church St, Ne...|   12|1700.0| New York City|   0|
|  1|  295666|  LG Washing Machine|               1|     600.0|2019-12-29 07:03:00|562 2nd St, New Y...|   12| 600.0| New York City|   7|
|  2|  295667|USB-C Charging Cable|               1|     11.95|2019-12-12 18:21:00|277 Main St, New ...|   12| 11.95| New York City|  18|
|  3|  295668|    27in FHD Monitor|               1|    149.99|2019-12-22 15:13:00|410 6th St, San F...|   12|149.99| San Francisco|  15|
|  4|  295669|USB-C Charging Cable